### Train, publish and use image detection model with Azure Custom Vision
This is first part which describes how to use API for upload taged images to Azure Custom Vision Studio project. Next part (az_image_detection2.ipynb) describes how to use API of trained model for object detection. <br>
Based on [this](https://microsoftlearning.github.io/mslearn-ai-vision/Instructions/Exercises/03-custom-vision-object-detection.html) Microsoft learn manual.

In [ ]:
%pip install azure-cognitiveservices-vision-customvision==3.1.1

In [1]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import time
import json
import os
from dotenv import load_dotenv

global training_client
global custom_vision_project

# Get Configuration Settings from .env file
load_dotenv()
training_endpoint = os.getenv('TRAINING_ENDPOINT')
training_key = os.getenv('TRAINING_KEY')
project_id = os.getenv('PROJECT_ID')

In [2]:
# Authenticate a client for the training API
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
training_client = CustomVisionTrainingClient(training_endpoint, credentials)

# Get the Custom Vision project previously created in Cusom Vision studio (https://www.customvision.ai/)
custom_vision_project = training_client.get_project(project_id)

In [11]:
# Get the tags defined in the project
tags = training_client.get_tags(custom_vision_project.id)

# Create a list of images with tagged regions
tagged_images_with_regions = []

# Get the images and tagged regions from the JSON file
url_json = os.path.join (os.getcwd(), "data", "training-images", "tagged-images.json")
with open(url_json, 'r') as json_file:
    tagged_images = json.load(json_file)
    for image in tagged_images['files']:
        # Get the filename
        file = image['filename']
        print("processing file: ", file)
        # Get the tagged regions
        regions = []
        for tag in image['tags']:
            tag_name = tag['tag']
            print("    adding tag: ", tag_name)
            # Look up the tag ID for this tag name
            tag_id = next(t for t in tags if t.name == tag_name).id
            # Add a region for this tag using the coordinates and dimensions in the JSON
            regions.append(Region(tag_id=tag_id, left=tag['left'],top=tag['top'],width=tag['width'],height=tag['height']))
        # Add the image and its regions to the list

        url_img = os.path.join (os.getcwd(), "data", "training-images", file)
        with open(url_img, mode="rb") as image_data:
            tagged_images_with_regions.append(ImageFileCreateEntry(name=file, contents=image_data.read(), regions=regions))


processing file:  image11.jpg
    adding tag:  orange
    adding tag:  banana
processing file:  image12.jpg
    adding tag:  banana
    adding tag:  aple
    adding tag:  orange
processing file:  image13.jpg
    adding tag:  orange
    adding tag:  banana
processing file:  image14.jpg
    adding tag:  aple
    adding tag:  banana
processing file:  image15.jpg
    adding tag:  orange
    adding tag:  banana
processing file:  image16.jpg
    adding tag:  banana
    adding tag:  aple
processing file:  image17.jpg
    adding tag:  banana
    adding tag:  aple
processing file:  image18.jpg
    adding tag:  banana
    adding tag:  orange
    adding tag:  orange
processing file:  image19.jpg
    adding tag:  orange
processing file:  image20.jpg
    adding tag:  orange
processing file:  image21.jpg
    adding tag:  orange
processing file:  image22.jpg
    adding tag:  orange
processing file:  image23.jpg
    adding tag:  orange
processing file:  image24.jpg
    adding tag:  banana
processing f

In [12]:
# Upload the list of images as a batch
upload_result = training_client.create_images_from_files(custom_vision_project.id, ImageFileCreateBatch(images=tagged_images_with_regions))
# Check for failure
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
else:
    print("Images uploaded.")

Images uploaded.


Now tagged images uploaded to the train project in Cusome Vision studio. <br>
Use Cusome Vision studio, train and publish model.
With next step we will use our trained model for image classification.